# Assess diversity of databases with Fairface

https://github.com/dchen236/FairFace?tab=readme-ov-file

In [ ]:
res34_fair_align_multi_4_20190809.pt
res34_fair_align_multi_7_20190809.pt

# Test dossier painting single face

In [5]:
import torch
import os
import pandas as pd
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from IPython.display import display  # Pour afficher les résultats dans Jupyter

# 📌 Définition des catégories FairFace (FairFace contient 18 classes)
ethnicities = [
    "White", "Black", "Latino_Hispanic", "East Asian", "Southeast Asian", 
    "Indian", "Middle Eastern", "Other", "Child", "Teen", "Young Adult", 
    "Adult", "Senior", "Male", "Female", "Smiling", "Not Smiling", "Unknown"
]

# 📌 Charger le modèle FairFace avec la bonne architecture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 18)  # Adapter à 18 classes

# 📌 Chemin du fichier modèle (⚠️ Change ce chemin si besoin)
model_path = "../FairFace/res34_fair_align_multi_4_20190809.pt"

# Vérifier si le fichier modèle existe
if not os.path.exists(model_path):
    raise FileNotFoundError(f"⚠️ Le fichier modèle {model_path} est introuvable. Vérifie le chemin.")

# Charger les poids du modèle FairFace
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Modèle FairFace chargé avec succès !")

# 🔹 Transformation des images (prétraitement pour le modèle)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def predict_ethnicity(image_path):
    """ Prédiction de l'ethnicité d'un visage dans une image """
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(image)
            prediction = torch.argmax(output, dim=1).item()

        return ethnicities[prediction]
    
    except Exception as e:
        print(f"⚠️ Erreur avec {image_path} : {e}")
        return "Unknown"

# 📌 Dossier contenant les images à analyser (⚠️ MODIFIER ICI)
image_folder = "../raw_data/painting_test_set"

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"⚠️ Le dossier {image_folder} est introuvable. Vérifie le chemin.")

# 🔹 Stocker les résultats
results = []

# Parcourir les images du dossier
for img_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_file)

    # Vérifier si c'est bien une image
    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        ethnicity = predict_ethnicity(img_path)
        results.append({"image": img_file, "ethnicity": ethnicity})
        print(f"✅ {img_file} → {ethnicity}")

# 🔹 Convertir en DataFrame et sauvegarder
df_results = pd.DataFrame(results)
df_results.to_csv("diversity_results.csv", index=False)

print("📊 Analyse terminée ! Résultats enregistrés dans diversity_results.csv")

# 🔹 Afficher les résultats dans Jupyter Notebook
display(df_results)


✅ Modèle FairFace chargé avec succès !
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg → White
✅ keisai-eisen_kiyomizu-komachi-1838.jpg → Latino_Hispanic
✅ andrÔö£ð¦Ôö¼ð¦-lhote_le-judgment-de-paris-1927.jpg → Senior
✅ benozzo-gozzoli_scenes-with-st-ambrose-detail-1465.jpg → Latino_Hispanic
✅ edouard-vuillard_self-portrait-1.jpg → Latino_Hispanic
✅ yiannis-tsaroychis_schizophrenic-1968.jpg → Latino_Hispanic
✅ antÔö£ð¦Ôö¼Ôöénio-de-carvalho-da-silva-porto_t-te-de-jeune-fille.jpg → East Asian
✅ juan-gris_pierrot-1919.jpg → White
✅ keisai-eisen_fukagawa-hachiman-no-shin-fuji-from-the-series-twelve-views-of-modern-beauties-imay-bijin-j-ni.jpg → White
✅ boris-kustodiev_portrait-of-a-singer-i-v-ershov-1905.jpg → Latino_Hispanic
✅ andy-warhol_muhammad-ali.jpg → Senior
✅ akseli-gallen-kallela_le-depart-de-v-in-m-inen-1906.jpg → White
✅ andrÔö£ð¦Ôö¼ð¦-lhote_les-deux-amies-1925.jpg → White
✅ antÔö£ð¦Ôö¼Ôöénio-de-carvalho-da-silva-porto_head-of-a-girl.jpg → East Asian
✅ fernand-leger_the-w

,image,ethnicity
0,sandro-botticelli_the-temptation-of-christ-148...,White
1,keisai-eisen_kiyomizu-komachi-1838.jpg,Latino_Hispanic
2,andrÔö£ð¦Ôö¼ð¦-lhote_le-judgment-de-paris-1927...,Senior
3,benozzo-gozzoli_scenes-with-st-ambrose-detail-...,Latino_Hispanic
4,edouard-vuillard_self-portrait-1.jpg,Latino_Hispanic
...,...,...
76,jamie-wyeth_portrait-of-andrew-wyeth-1969.jpg,Black
77,jacob-jordaens_as-the-old-sang-so-the-young-pi...,Other
78,juan-gris_pierrot-with-guitar-1925.jpg,White
79,martial-raysse_portrait-de-mme-raysse-1963.jpg,White


# Prise en compte multi-visages

In [16]:
import torch
import os
import pandas as pd
import cv2
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from IPython.display import display

# 📌 Définition des catégories FairFace
ethnicities = [
    "White", "Black", "Latino_Hispanic", "East Asian", "Southeast Asian", 
    "Indian", "Middle Eastern", "Other", "Child", "Teen", "Young Adult", 
    "Adult", "Senior", "Male", "Female", "Smiling", "Not Smiling", "Unknown"
]

# 📌 Charger le modèle FairFace
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 18)  # Adapter à 18 classes

# 📌 Chemin du fichier modèle (⚠️ Change ce chemin si besoin)
model_path = "../FairFace/res34_fair_align_multi_7_20190809.pt"

# Vérifier si le fichier modèle existe
if not os.path.exists(model_path):
    raise FileNotFoundError(f"⚠️ Le fichier modèle {model_path} est introuvable. Vérifie le chemin.")

# Charger les poids du modèle FairFace
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Modèle FairFace chargé avec succès !")

# 🔹 Transformation des images (prétraitement pour le modèle)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🔹 Charger un détecteur de visages avec OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_faces(image_path):
    """ Détecte les visages dans une image et retourne une liste de visages recadrés """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Détecter les visages
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Extraire chaque visage détecté
    face_crops = []
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        face_crops.append(face)

    return face_crops

def predict_ethnicity(face_image):
    """ Prédiction de l'ethnicité d'un visage recadré """
    try:
        # Convertir l'image en PIL
        face_pil = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
        face_pil = transform(face_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(face_pil)
            prediction = torch.argmax(output, dim=1).item()

        return ethnicities[prediction]

    except Exception as e:
        print(f"⚠️ Erreur avec un visage : {e}")
        return "Unknown"

# 📌 Dossier contenant les images à analyser (⚠️ MODIFIER ICI)
image_folder = "../raw_data/painting_test_set"

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise FileNotFoundError(f"⚠️ Le dossier {image_folder} est introuvable. Vérifie le chemin.")

# 🔹 Stocker les résultats
results = []

# Parcourir les images du dossier
for img_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_file)

    # Vérifier si c'est bien une image
    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        face_crops = detect_faces(img_path)
        
        if len(face_crops) == 0:
            print(f"⚠️ Aucun visage détecté dans {img_file}")
            results.append({"image": img_file, "face_id": "None", "ethnicity": "Unknown"})
        else:
            for i, face in enumerate(face_crops):
                ethnicity = predict_ethnicity(face)
                results.append({"image": img_file, "face_id": f"Face {i+1}", "ethnicity": ethnicity})
                print(f"✅ {img_file} - Face {i+1} → {ethnicity}")

# 🔹 Convertir en DataFrame et sauvegarder
df_results = pd.DataFrame(results)
df_results.to_csv("diversity_results.csv", index=False)

print("📊 Analyse terminée ! Résultats enregistrés dans diversity_results.csv")

# 🔹 Afficher les résultats dans Jupyter Notebook
display(df_results)


✅ Modèle FairFace chargé avec succès !
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 1 → White
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 2 → Senior
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 3 → Middle Eastern
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 4 → Senior
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 5 → Senior
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 6 → Black
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 7 → Senior
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 1 → Middle Eastern
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 2 → White
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 3 → Middle Eastern
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 4 → Senior
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 5 → East Asian
✅ andrÔö£ð¦Ôö¼ð¦-lhote_le-judgment-de-paris-1927.jpg - Face 1 → Middle Eastern
✅ andrÔö£ð¦Ôö¼ð¦-lhote_le-judgment-de

,image,face_id,ethnicity
0,sandro-botticelli_the-temptation-of-christ-148...,Face 1,White
1,sandro-botticelli_the-temptation-of-christ-148...,Face 2,Senior
2,sandro-botticelli_the-temptation-of-christ-148...,Face 3,Middle Eastern
3,sandro-botticelli_the-temptation-of-christ-148...,Face 4,Senior
4,sandro-botticelli_the-temptation-of-christ-148...,Face 5,Senior
...,...,...,...
232,martial-raysse_portrait-de-mme-raysse-1963.jpg,Face 1,White
233,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 1,Senior
234,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 2,White
235,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 3,Senior


In [17]:
df_results["ethnicity"].value_counts()

ethnicity
White              75
Senior             64
Middle Eastern     28
Black              15
Male               15
East Asian         13
Unknown            12
Other               5
Not Smiling         2
Adult               2
Young Adult         2
Southeast Asian     1
Latino_Hispanic     1
Indian              1
Female              1
Name: count, dtype: int64

# Prise en compte multi-visages avec labels sur la photo

In [19]:
import torch
import os
import pandas as pd
import cv2
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
from IPython.display import display

# 📌 Définition des 18 catégories FairFace
ethnicities = [
    "White", "Black", "Latino_Hispanic", "East Asian", "Southeast Asian", 
    "Indian", "Middle Eastern", "Other", "Child", "Teen", "Young Adult", 
    "Adult", "Senior", "Male", "Female", "Smiling", "Not Smiling", "Unknown"
]

# 📌 Charger le modèle FairFace
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(pretrained=False)

# 📌 Garder la dernière couche d'origine avec 18 classes
model.fc = torch.nn.Linear(model.fc.in_features, 18)

# 📌 Chemin du fichier modèle (⚠️ Modifier ici si nécessaire)
model_path = "../FairFace/res34_fair_align_multi_4_20190809.pt"

if not os.path.exists(model_path):
    raise FileNotFoundError(f"⚠️ Le fichier modèle {model_path} est introuvable.")

# Charger les poids du modèle (pré-entraîné avec 18 classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Modèle FairFace chargé avec succès avec 18 catégories !")

# 🔹 Transformation des images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🔹 Détecteur de visages OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_faces(image_path):
    """ Détecte les visages et retourne une liste des coordonnées (x, y, w, h) """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def predict_ethnicity(face_image):
    """ Prédiction de l'ethnicité d'un visage """
    try:
        face_pil = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
        face_pil = transform(face_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(face_pil)
            prediction = torch.argmax(output, dim=1).item()

        return ethnicities[prediction]  # 📌 Associer l'indice à l'ethnicité correspondante
    except Exception as e:
        print(f"⚠️ Erreur avec un visage : {e}")
        return "Unknown"

# 📌 Dossier des images (⚠️ Modifier ici)
image_folder = "../raw_data/painting_test_set"
output_folder = "../processed_data/FairFace"
os.makedirs(output_folder, exist_ok=True)

if not os.path.exists(image_folder):
    raise FileNotFoundError(f"⚠️ Le dossier {image_folder} est introuvable.")

# 🔹 Stocker les résultats
results = []

# 🔹 Traitement des images
for img_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_file)

    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image = cv2.imread(img_path)
        faces = detect_faces(img_path)
        
        if len(faces) == 0:
            print(f"⚠️ Aucun visage détecté dans {img_file}")
            results.append({"image": img_file, "face_id": "None", "ethnicity": "Unknown", "x": None, "y": None, "w": None, "h": None})
        else:
            for i, (x, y, w, h) in enumerate(faces):
                face = image[y:y+h, x:x+w]
                ethnicity = predict_ethnicity(face)

                results.append({"image": img_file, "face_id": f"Face {i+1}", "ethnicity": ethnicity, "x": x, "y": y, "w": w, "h": h})
                print(f"✅ {img_file} - Face {i+1} → {ethnicity} (x={x}, y={y}, w={w}, h={h})")

                # 🖼 Annoter l'image
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, ethnicity, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Sauvegarder l'image annotée
            annotated_path = os.path.join(output_folder, f"annotated_{img_file}")
            cv2.imwrite(annotated_path, image)

# 🔹 Convertir en DataFrame et sauvegarder
df_results = pd.DataFrame(results)
df_results.to_csv("face_identification_results.csv", index=False)

print("📊 Analyse terminée ! Résultats enregistrés dans face_identification_results.csv")

# 🔹 Afficher les résultats dans Jupyter Notebook
display(df_results)


/home/carozero/.pyenv/versions/3.10.6/envs/delorean_art/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/carozero/.pyenv/versions/3.10.6/envs/delorean_art/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Modèle FairFace chargé avec succès avec 18 catégories !
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 1 → White (x=123, y=494, w=60, h=60)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 2 → Black (x=847, y=756, w=69, h=69)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 3 → Black (x=1477, y=766, w=73, h=73)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 4 → White (x=1859, y=879, w=173, h=173)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 5 → White (x=1244, y=1151, w=49, h=49)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 6 → White (x=1355, y=1119, w=165, h=165)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 7 → White (x=292, y=1299, w=60, h=60)
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 1 → Latino_Hispanic (x=1156, y=681, w=50, h=50)
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 2 → White (x=430, y=607, w=73, h=73)
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - 

,image,face_id,ethnicity,x,y,w,h
0,sandro-botticelli_the-temptation-of-christ-148...,Face 1,White,123.0,494.0,60.0,60.0
1,sandro-botticelli_the-temptation-of-christ-148...,Face 2,Black,847.0,756.0,69.0,69.0
2,sandro-botticelli_the-temptation-of-christ-148...,Face 3,Black,1477.0,766.0,73.0,73.0
3,sandro-botticelli_the-temptation-of-christ-148...,Face 4,White,1859.0,879.0,173.0,173.0
4,sandro-botticelli_the-temptation-of-christ-148...,Face 5,White,1244.0,1151.0,49.0,49.0
...,...,...,...,...,...,...,...
232,martial-raysse_portrait-de-mme-raysse-1963.jpg,Face 1,White,501.0,418.0,611.0,611.0
233,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 1,White,358.0,448.0,78.0,78.0
234,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 2,White,448.0,306.0,377.0,377.0
235,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 3,White,1209.0,862.0,77.0,77.0


## Limitation aux 6 premières cat

In [20]:
import torch
import os
import pandas as pd
import cv2
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
from IPython.display import display

# 📌 Définition des 6 catégories FairFace que l'on veut garder
ethnicities = [
    "White", "Black", "Latino_Hispanic", "East Asian", "Southeast Asian", 
    "Indian"
]

# 📌 Charger le modèle FairFace
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(pretrained=False)

# 📌 Ne PAS modifier la couche fc, on garde les 18 classes
model.fc = torch.nn.Linear(model.fc.in_features, 18)

# 📌 Chemin du fichier modèle (⚠️ Modifier ici si nécessaire)
model_path = "../FairFace/res34_fair_align_multi_4_20190809.pt"

if not os.path.exists(model_path):
    raise FileNotFoundError(f"⚠️ Le fichier modèle {model_path} est introuvable.")

# Charger les poids du modèle (pré-entraîné avec 18 classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print("✅ Modèle FairFace chargé avec succès avec 18 catégories, en filtrant sur 6 catégories.")

# 🔹 Transformation des images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🔹 Détecteur de visages OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_faces(image_path):
    """ Détecte les visages et retourne une liste des coordonnées (x, y, w, h) """
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

def predict_ethnicity(face_image):
    """ Prédiction de l'ethnicité d'un visage (en filtrant sur 6 catégories) """
    try:
        face_pil = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
        face_pil = transform(face_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(face_pil)  # Sortie avec 18 classes
            output = output[:,:6]  # 📌 On garde seulement les 6 premières classes
            prediction = torch.argmax(output, dim=1).item()

        return ethnicities[prediction]  # Associer l'indice à l'ethnicité correspondante
    except Exception as e:
        print(f"⚠️ Erreur avec un visage : {e}")
        return "Unknown"

# 📌 Dossier des images (⚠️ Modifier ici)
image_folder = "../raw_data/painting_test_set"
output_folder = "../processed_data/FairFace"
os.makedirs(output_folder, exist_ok=True)

if not os.path.exists(image_folder):
    raise FileNotFoundError(f"⚠️ Le dossier {image_folder} est introuvable.")

# 🔹 Stocker les résultats
results = []

# 🔹 Traitement des images
for img_file in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_file)

    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        image = cv2.imread(img_path)
        faces = detect_faces(img_path)
        
        if len(faces) == 0:
            print(f"⚠️ Aucun visage détecté dans {img_file}")
            results.append({"image": img_file, "face_id": "None", "ethnicity": "Unknown", "x": None, "y": None, "w": None, "h": None})
        else:
            for i, (x, y, w, h) in enumerate(faces):
                face = image[y:y+h, x:x+w]
                ethnicity = predict_ethnicity(face)

                results.append({"image": img_file, "face_id": f"Face {i+1}", "ethnicity": ethnicity, "x": x, "y": y, "w": w, "h": h})
                print(f"✅ {img_file} - Face {i+1} → {ethnicity} (x={x}, y={y}, w={w}, h={h})")

                # 🖼 Annoter l'image
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, ethnicity, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Sauvegarder l'image annotée
            annotated_path = os.path.join(output_folder, f"annotated_{img_file}")
            cv2.imwrite(annotated_path, image)

# 🔹 Convertir en DataFrame et sauvegarder
df_results = pd.DataFrame(results)
df_results.to_csv("face_identification_results_6_classes.csv", index=False)

print("📊 Analyse terminée ! Résultats enregistrés dans face_identification_results_6_classes.csv")

# 🔹 Afficher les résultats dans Jupyter Notebook
display(df_results)


/home/carozero/.pyenv/versions/3.10.6/envs/delorean_art/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/carozero/.pyenv/versions/3.10.6/envs/delorean_art/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


✅ Modèle FairFace chargé avec succès avec 18 catégories, en filtrant sur 6 catégories.
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 1 → White (x=123, y=494, w=60, h=60)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 2 → Black (x=847, y=756, w=69, h=69)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 3 → Black (x=1477, y=766, w=73, h=73)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 4 → White (x=1859, y=879, w=173, h=173)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 5 → White (x=1244, y=1151, w=49, h=49)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 6 → White (x=1355, y=1119, w=165, h=165)
✅ sandro-botticelli_the-temptation-of-christ-1482(1).jpg - Face 7 → White (x=292, y=1299, w=60, h=60)
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 1 → Latino_Hispanic (x=1156, y=681, w=50, h=50)
✅ keisai-eisen_kiyomizu-komachi-1838.jpg - Face 2 → White (x=430, y=607, w=73, h=73)
✅ keisai-eisen

,image,face_id,ethnicity,x,y,w,h
0,sandro-botticelli_the-temptation-of-christ-148...,Face 1,White,123.0,494.0,60.0,60.0
1,sandro-botticelli_the-temptation-of-christ-148...,Face 2,Black,847.0,756.0,69.0,69.0
2,sandro-botticelli_the-temptation-of-christ-148...,Face 3,Black,1477.0,766.0,73.0,73.0
3,sandro-botticelli_the-temptation-of-christ-148...,Face 4,White,1859.0,879.0,173.0,173.0
4,sandro-botticelli_the-temptation-of-christ-148...,Face 5,White,1244.0,1151.0,49.0,49.0
...,...,...,...,...,...,...,...
232,martial-raysse_portrait-de-mme-raysse-1963.jpg,Face 1,White,501.0,418.0,611.0,611.0
233,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 1,White,358.0,448.0,78.0,78.0
234,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 2,White,448.0,306.0,377.0,377.0
235,john-french-sloan_jeanne-dubinsky-1942.jpg,Face 3,White,1209.0,862.0,77.0,77.0
